In [220]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd

In [221]:
#Save Credentials
with open('SpotifyID.txt', 'r') as f:
    client_id = f.read().replace('\n','')
with open('SpotifySecret.txt', 'r') as f:
    client_secret = f.read().replace('\n','')


#setup spotipy
username= "225a5fp37icg43v4vhnztptfy"    
scope = 'user-library-read playlist-modify-public playlist-read-private'
redirect_uri='https://developer.spotify.com/dashboard/applications/ec25c1dfa01e4712b0d4d234f8b51a32'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [222]:
def search_playlist(sp,new_playlist):
    playListID =""
    playlists = sp.user_playlists(username)
    for playlist in playlists["items"]:
        if (playlist["name"]==new_playlist):
            playListID = playlist["id"]
    return playListID

In [223]:
def create_playlist(new_playlist,sp):
    
    #See if playlist already exists
    #if exists, return playlist id
    playListID = search_playlist(sp,new_playlist)
    
    if (playListID==""):
        sp.user_playlist_create(username, new_playlist,public = True)
        playListID = search_playlist(sp,new_playlist)
    return playListID
    

In [224]:
def get_bag_of_songs(songs,cities):
    return songs.loc[songs['city'].isin(cities)]

In [225]:
def add_songs(playListID,songs,duration,cities,sp):
    bag_of_songs = get_bag_of_songs(songs,cities)
    
    while duration>0:
        uri = bag_of_songs.sample(weights = bag_of_songs.groupby('city')['city'].transform('count')).iloc[0,-4]
        song_length = bag_of_songs.sample(weights = bag_of_songs.groupby('city')['city'].transform('count')).iloc[0,-3]
        if uri!="error":
            #print(uri)
            sp.user_playlist_add_tracks(username, playlist_id = playListID, tracks=[uri], position=None)
        duration = duration-song_length
    

In [226]:
playListID = create_playlist("roadtrip2",sp)
duration = 30*60*1000
cities = ["New York City", "Birmingham","Los Angeles"]
songs = pd.read_csv(".\datasets\songs.csv")
bag_of_songs = songs.loc[songs['city'].isin(cities)]
add_songs(playListID,songs,duration,cities,sp)